In [73]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
from fits_align.ident import make_transforms
from astropy.io import fits
from astropy import wcs
from astropy.units import degree
from astropy.coordinates import SkyCoord

In [74]:
indir = '/Users/egomez/Downloads/lcogtdata-20200116-213/'

In [75]:
img_list = glob.glob('/Users/egomez/Downloads/lcogtdata-20200116-213/*.fz') 

Make a large ndarray of all the data, sorted by flux

In [76]:
def data_sort(image):
    with fits.open(image) as hdul:
        b = np.sort(hdul[2].data, order=['flux']) 
        w = wcs.WCS(hdul[1].header)
        ra, dec = w.wcs_pix2world(b['x'], b['y'], 1)
        med = np.median(b['flux'])
        a = b[b['flux']>0.5*med][::-1]
        counts = a['flux']
        countserr = a['fluxerr']
        ref_a = np.array([a['flux'],a['x'], a['y']])
    return ref_a

Align the arrays

In [77]:
ref_image = img_list[0] 
images_to_align = img_list[1:] 
identifications = make_transforms(ref_image, images_to_align)

In [78]:
# dt = [('flux',tmpa.dtype), ('x', tmpa.dtype)]
# assert tmpa.flags['C_CONTIGUOUS']
# phot_ref = tmpa.ravel().view(dt)
# phot_ref.sort(order=['flux','x'])

In [79]:
photarray = []
old = []
for id in identifications:
    a = data_sort(id.ukn.filepath)
    coords = [a[1],a[2]]
    inv = id.trans.inverse()
    (matrix, offset) = inv.matrixform()
    xy = np.dot(matrix,coords) - offset.reshape(2,1)
#         ra, dec = w.wcs_pix2world(xy[0], xy[1], 1)
    tmpa = np.array([a[0],xy[0],xy[1]])
    photarray.append(tmpa)

Set OBSGEO-B to    28.300308 from OBSGEO-[XYZ].
Set OBSGEO-H to     2386.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [86]:
a = data_sort(ref_image)
b = photarray[10].copy()
m = min(a.shape, b.shape)[1]

In [81]:
pa_align = []

Check if some of the coordinates are switch (because the fluxes were close)

In [82]:
m_tot = 10000
for b in photarray:
    b_t = b.T
    m = min(a.shape, b.shape)[1]
    m_tot = m if m < m_tot else m_tot
    c = abs(a.T[:m]-b.T[:m])
    for ind in range(2,m-1):
        if c[ind,1]> 3:
            for ia in range(0,4):
                if np.abs(a.T[ind]-b_t[ind-ia])[1] < 3 and np.abs(a.T[ind]-b_t[ind-ia])[2] < 3:
                    b_t[[ind,ind-ia]] = b_t[[ind-ia,ind]]
                    continue
    pa_align.append(b_t)
# How close are the 2 arrays to matching

Make a map of the images to use by comparing the coordinate columns in each frame with the reference frame. Remove images which have less than 70 matching stars because these probably have things wrong with them.

In [90]:
cl = np.ones((m_tot, 3), dtype=bool)
count =0
phot = [a]
for b in pa_align:
    if np.sum(np.isclose(a[1,:m_tot], b.T[1,:m_tot], atol=5)) < 70:
        continue
    count +=1
    phot.append(b.T)
    cl *= np.isclose(a.T[:m_tot], b[:m_tot], atol=5)
print(f"{np.sum(cl.T[1])}:{np.sum(cl.T[2])} comparison stars")
print(f"{count/len(pa_align):.2f} % of files used")

27:27 comparison stars
0.79 % of files used


In [84]:
# for i in range(0,len(photarray)):
#      plt.scatter(photarray[i],marker='.',c='blue',alpha=0.005)
plt.scatter(phot_ref,phot_ref,marker='.',c='red',alpha=0.5)

NameError: name 'phot_ref' is not defined